In [0]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [0]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
from google.colab import files
files.upload()

Saving Wireless_Telecom_1.csv to Wireless_Telecom_1.csv


{'Wireless_Telecom_1.csv': b'Sensor 1,Sensor 2,Sensor 3,Sensor 4\r\n-0.85714,-0.52,0.95238,0.35\r\n-1,-0.76,0.71429,0.35\r\n-1,-0.8,0.85714,0.35\r\n-1,-0.44,0.71429,0.55\r\n-1,-0.32,0.7619,0.65\r\n-1,-0.56,0.95238,0.65\r\n-1,-0.6,0.90476,0.8\r\n-1,-0.72,0.80952,0.9\r\n-0.80952,-0.52,0.47619,0.8\r\n-0.66667,-0.6,0.42857,0.8\r\n-1,-0.28,0.14286,0.5\r\n-0.71429,-0.2,0.095238,0.45\r\n-0.52381,-0.08,0.095238,0.65\r\n-0.47619,-0.44,0.28571,0.55\r\n-0.28571,-0.44,0.095238,0.15\r\n-0.2381,-0.08,-0.19048,-0.15\r\n-0.2381,-0.12,0.14286,0.2\r\n-0.28571,-0.44,-0.14286,0.35\r\n-0.28571,-0.44,-0.095238,0\r\n-0.19048,-0.44,0.19048,0.15\r\n-0.14286,-0.16,-0.14286,0.2\r\n-0.28571,-0.04,0,0.2\r\n-0.095238,0.04,0.14286,-0.4\r\n0,-0.08,0,0.05'}

In [0]:
df1 = pd.read_csv('Wireless_Telecom_1.csv')



In [5]:
df1.head()


,Sensor 1,Sensor 2,Sensor 3,Sensor 4
0,-0.85714,-0.52,0.95238,0.35
1,-1.00000,-0.76,0.71429,0.35
2,-1.00000,-0.80,0.85714,0.35
3,-1.00000,-0.44,0.71429,0.55
4,-1.00000,-0.32,0.76190,0.65


In [6]:
df1.shape

(24, 4)

In [8]:
path = '/content/Wireless_Telecom_'
sequences = list()
for i in range(1,22):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

/content/Wireless_Telecom_1.csv
/content/Wireless_Telecom_2.csv
/content/Wireless_Telecom_3.csv
/content/Wireless_Telecom_4.csv
/content/Wireless_Telecom_5.csv
/content/Wireless_Telecom_6.csv
/content/Wireless_Telecom_7.csv
/content/Wireless_Telecom_8.csv
/content/Wireless_Telecom_9.csv
/content/Wireless_Telecom_10.csv
/content/Wireless_Telecom_11.csv
/content/Wireless_Telecom_12.csv
/content/Wireless_Telecom_13.csv
/content/Wireless_Telecom_14.csv
/content/Wireless_Telecom_15.csv
/content/Wireless_Telecom_16.csv
/content/Wireless_Telecom_17.csv
/content/Wireless_Telecom_18.csv
/content/Wireless_Telecom_19.csv
/content/Wireless_Telecom_20.csv
/content/Wireless_Telecom_21.csv


In [7]:
pwd

'/content'

In [0]:
targets = pd.read_csv('/content/Wireless_Telecom_target.csv')
targets = targets.values[:,1]

In [10]:
sequences[0]

array([[-0.85714 , -0.52    ,  0.95238 ,  0.35    ],
       [-1.      , -0.76    ,  0.71429 ,  0.35    ],
       [-1.      , -0.8     ,  0.85714 ,  0.35    ],
       [-1.      , -0.44    ,  0.71429 ,  0.55    ],
       [-1.      , -0.32    ,  0.7619  ,  0.65    ],
       [-1.      , -0.56    ,  0.95238 ,  0.65    ],
       [-1.      , -0.6     ,  0.90476 ,  0.8     ],
       [-1.      , -0.72    ,  0.80952 ,  0.9     ],
       [-0.80952 , -0.52    ,  0.47619 ,  0.8     ],
       [-0.66667 , -0.6     ,  0.42857 ,  0.8     ],
       [-1.      , -0.28    ,  0.14286 ,  0.5     ],
       [-0.71429 , -0.2     ,  0.095238,  0.45    ],
       [-0.52381 , -0.08    ,  0.095238,  0.65    ],
       [-0.47619 , -0.44    ,  0.28571 ,  0.55    ],
       [-0.28571 , -0.44    ,  0.095238,  0.15    ],
       [-0.2381  , -0.08    , -0.19048 , -0.15    ],
       [-0.2381  , -0.12    ,  0.14286 ,  0.2     ],
       [-0.28571 , -0.44    , -0.14286 ,  0.35    ],
       [-0.28571 , -0.44    , -0.095238,  0.  

In [74]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()


count    21.000000
mean     25.809524
std       2.204973
min      23.000000
25%      24.000000
50%      25.000000
75%      27.000000
max      30.000000
dtype: float64

In [52]:
len_sequences

[24,
 24,
 24,
 24,
 24,
 24,
 23,
 27,
 28,
 24,
 26,
 29,
 29,
 25,
 30,
 27,
 29,
 23,
 25,
 27,
 26]

In [0]:
#Padding the sequence with the values in last row to max length
to_pad = 30
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 24
from keras.preprocessing import sequence
seq_len = 24
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [0]:
train = [final_seq[i] for i in range(1,16)]
test = [final_seq[i] for i in range(17,21)]

train_target = [targets[i] for i in range(1,16)]
test_target = [targets[i] for i in range(17,21)]

In [16]:
train

[array([-0.7619 , -0.28   ,  0.28571,  0.35   ]),
 array([-0.52381, -0.08   ,  0.     ,  0.35   ]),
 array([-0.52381 , -0.72    ,  0.095238,  0.45    ]),
 array([-0.52381 , -0.32    ,  0.047619,  0.25    ]),
 array([-0.7619 , -0.32   ,  0.14286,  0.6    ]),
 array([-1.     , -0.84   ,  0.19048,  0.75   ]),
 array([-0.85714, -0.68   ,  0.57143, -0.6    ]),
 array([-0.80952, -0.56   ,  0.28571,  0.4    ]),
 array([-0.71429 , -0.52    ,  0.047619,  0.5     ]),
 array([-0.90476, -0.72   ,  0.     ,  0.55   ]),
 array([-1.  , -0.36,  0.  ,  0.3 ]),
 array([-1.     , -0.76   , -0.19048,  0.2    ]),
 array([-0.80952, -0.52   ,  0.     ,  0.65   ]),
 array([-0.80952, -0.76   , -0.28571,  0.3    ]),
 array([-0.66667, -0.28   ,  0.     ,  0.1    ])]

In [0]:
train = np.array(train)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))

In [49]:
final_seq

array([[[-0.85714 , -0.52    ,  0.95238 ,  0.35    ],
        [-1.      , -0.76    ,  0.71429 ,  0.35    ],
        [-1.      , -0.8     ,  0.85714 ,  0.35    ],
        ...,
        [-0.28571 , -0.04    ,  0.      ,  0.2     ],
        [-0.095238,  0.04    ,  0.14286 , -0.4     ],
        [ 0.      , -0.08    ,  0.      ,  0.05    ]],

       [[-0.57143 , -0.68    ,  0.85714 ,  0.25    ],
        [-1.      , -0.44    ,  0.57143 ,  0.5     ],
        [-0.61905 , -0.68    ,  0.90476 ,  0.5     ],
        ...,
        [-0.80952 , -0.28    , -0.14286 ,  0.      ],
        [-0.57143 , -0.32    , -0.61905 ,  0.2     ],
        [-0.33333 , -0.32    , -0.28571 , -0.25    ]],

       [[-1.      , -0.64    ,  0.28571 ,  0.35    ],
        [-0.61905 , -0.2     ,  0.71429 ,  0.3     ],
        [-0.7619  , -0.32    ,  0.42857 ,  0.25    ],
        ...,
        [-0.14286 , -0.08    , -0.047619,  0.1     ],
        [-0.2381  , -0.16    , -0.095238,  0.35    ],
        [-0.19048 , -0.08    ,  0.14286

In [50]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(test,test_target))




Train on 15 samples, validate on 4 samples
Epoch 1/200





15/15 [==============================] - 2s 105ms/step - loss: 0.7153 - acc: 0.3333 - val_loss: 0.6935 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to best_model.pkl
Epoch 2/200
15/15 [==============================] - 0s 6ms/step - loss: 0.6728 - acc: 0.8000 - val_loss: 0.7476 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.50000
Epoch 3/200
15/15 [==============================] - 0s 5ms/step - loss: 0.6358 - acc: 0.6667 - val_loss: 0.8144 - val_acc: 0.0000e+00

Epoch 00003: val_acc did not improve from 0.50000
Epoch 4/200
15/15 [==============================] - 0s 5ms/step - loss: 0.5998 - acc: 0.6667 - val_loss: 0.9119 - val_acc: 0.0000e+00

Epoch 00004: val_acc did not improve from 0.50000
Epoch 5/200
15/15 [==============================] - 0s 5ms/step - loss: 0.5620 - acc: 0.6667 - val_loss: 1.0795 - val_acc: 0.0000e+00

Epoch 00005: val_acc did not improve f

In [51]:
#loading the model and checking accuracy on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

1.0